In [1]:
import requests as rq
import pandas as pd
import io

In [2]:
strtDd = "20190101"
endDd = "20250516"
isuCd = "KR7443060009"

In [3]:
headers = {
    'Referer': "http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020203",
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36"
}

otp_url ='http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd' 

otp_form_data = {
  'locale': 'ko_KR',
  "share": '1',
  "csvxls_isNo": 'false',
  "name": 'fileDown',
  "url": 'dbms/MDC/STAT/standard/MDCSTAT01701',
  'adjStkPrc': 2,
  'adjStkPrc_check': 'Y',
  'strtDd': strtDd,
  'endDd': endDd,
  'isuCd': isuCd
}

otp = rq.get(otp_url, otp_form_data, headers=headers)

download_url = "http://data.krx.co.kr/comm/fileDn/download_excel/download.cmd"
download_form_data = {
    'code': otp.content
}

res = rq.post(download_url, download_form_data, headers=headers)
df = pd.read_excel(io.BytesIO(res.content))
df['일자'] = pd.to_datetime(df['일자'])

c:\Users\siyun\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [4]:
# 각 분기의 마지막 날짜 데이터만 필터링
quarterly_data = df.copy()
quarterly_data['연월'] = quarterly_data['일자'].dt.to_period('Q')  # 분기(Quarter) 기준으로 기간 설정
quarterly_data = quarterly_data.sort_values('일자')
quarterly_data = quarterly_data.drop_duplicates(subset=['연월'], keep='last')  # 각 분기의 마지막 데이터만 유지

# 필요한 컬럼만 선택
result = quarterly_data[['일자', '종가', '시가총액', '상장주식수']].T
result

,212,150,89,31,0
일자,2024-06-28 00:00:00,2024-09-30 00:00:00,2024-12-30 00:00:00,2025-03-31 00:00:00,2025-05-16 00:00:00
종가,130000,103700,161400,133800,163000
시가총액,5778500000000,4609465000000,7174230000000,5998148298000,7311644151000
상장주식수,44450000,44450000,44450000,44829210,44829210


In [5]:
result.to_excel(f'./price/{isuCd}_{strtDd}-{endDd}.xlsx', engine='openpyxl', header=False)